![QuantConnect Logo](https://cdn.quantconnect.com/web/i/logo-small.png)
# QuantConnect Alpha Streams
#### Licensed Under QuantConnect Platform Service Agreement v1.0. Copyright QuantConnect 2018.
#### For more information; see the Open API Specification of Alpha Streams: https://app.swaggerhub.com/apis/QuantConnect/Alpha/0.4

## Introduction

QuantConnect Alpha Streams allows quantiative funds to search a database of algorithm authors; and instantly license the authors' strategies (`Alpha`). The license agreements and fee structure is already prenegotiated, and the standardized backtesting and live trading infrastructure can give funds confidence in the algorithm results. Upon licensing an Alpha the insights/signals are instantly delivered to the fund in realtime.

## Terminology
 - `Alpha`     - Strategy for licensing.
 - `Author`    - QuantConnect member and creator of the Alpha. 
 - `Insight`   - Trade signal from an algorithm; consisting of an asset, direction, magnitude and confidence.
 - `Project`   - Properties of the project which submitted the `Alpha` 

## SDK Demonstration


### 1. Setup the SDK: Login with Client Credentials
We have written a full abstraction and SDK for the API which you can quickly import into your investment process. To start create a new instance of the `AlphaStreamClient` with your clientId and token.

In [1]:
# Import Python SDK Client Library 
from AlphaStreamClient import AlphaStreamClient 

# Define the clientId and API access token:
clientId    = "c7bd966e930c4b15b2ec13eb0d6170d9"
token       = "7030e89cfcc1948f4f93e91edd93d6f687c737844a6969d99d609a78f8d0a5c4091ef11f31c4c0e9cccacefe36ff4c2ad0e15525a85c65b0eafa34064cd11b1c"

# Create a new instance of the client module:
client = AlphaStreamClient(clientId, token)

### 2. Searching Alpha Strategies For Licensing
The primary aim of the Alpha Streams API is to quickly search for Alpha which matches your criteria. This criteria is unique to your fund and might include such aspects as: asset class, time horizon, risk adjusted return or market depth. Alpha Streams API aims to give you flexibility you need to locate suitable algorithms.

In [2]:
# Import Alpha Search Request Requirements
from Requests.SearchAlphasRequest import SearchAlphasRequest

# Define search criteria
projectId = 830918

# Submit Search:
alphas = client.SearchAlphas( projectId = projectId, sharedFeeMinimum = 1000 )

# Display some information on the algorithms:
for a in alphas:
    print("Alpha: id: {} Project: '{}' Fee: ${:,.2f}/mo".format(a.Id, a.Project.Name, a.SharedSubscriptionFee))

### 3. Pull Information on an Alpha By Specific AlphaId
You can fetch information on an alpha by its associated id with the `/alpha/{id}` API endpoint.

In [4]:
# Prospective / Test Alpha Id:
alphaId = "392a40ccab3740287a1c30bc6"
alpha = client.GetAlphaById( alphaId )

# Print out the Alpha data:
print( "Alpha Matched Id: \n\t id: {} \n\t Project: {} \n\t ExclusiveAvailable: {} \n\t Listed: {} ".
      format(alpha.Id, alpha.Project.Name, alpha.ExclusiveAvailable, alpha.ListedTime) )

Alpha Matched Id: 
	 id: 392a40ccab3740287a1c30bc6 
	 Project: DropShot Alpha Stream 40-50mm 
	 ExclusiveAvailable: True 
	 Listed: 2018-07-23 16:53:04 


### 4. Get Last 5 Insights of an Alpha
All historical insights of an Alpha are recorded in a database. Insights are generated `in sample`, `out of sample` or in `live trading`. Using these insights you can perform your own backtests with your own fill and slippage models to see how the strategy would have performed with your execution.

In [10]:
# Import the Insights Request Class:
from Requests.GetAlphaInsightsRequest import GetAlphaInsightsRequest

# Request Insights for the Previously Alpha
insights = client.GetAlphaInsights( alphaId, start = 10000 )

# Take Top 5:
for i in insights[:10]:
    print( "{} Alpha {} {} insight for {:<10} going {} by {:.2%} over the next {}s".format(i.CreatedTime, i.Source, i.Type, i.Ticker, i.Direction, i.Magnitude, i.Period) )

2009-09-25 19:45:00 Alpha in sample price insight for xlf        going up by 0.00% over the next 86400s
2010-07-23 19:45:00 Alpha in sample price insight for xlf        going up by 0.00% over the next 86400s
2018-03-15 19:45:00 Alpha in sample price insight for xlf        going up by 0.00% over the next 86400s
2016-10-17 19:45:00 Alpha in sample price insight for xlf        going up by 0.00% over the next 86400s
2011-05-23 19:45:00 Alpha in sample price insight for xlf        going up by 0.00% over the next 86400s
2012-07-10 19:45:00 Alpha in sample price insight for xlf        going up by 0.00% over the next 86400s
2017-06-26 19:45:00 Alpha in sample price insight for xlf        going up by 0.00% over the next 86400s
2008-12-01 20:45:00 Alpha in sample price insight for xlf        going up by 0.00% over the next 86400s
2015-01-16 20:45:00 Alpha in sample price insight for xlf        going up by 0.00% over the next 86400s
2009-09-08 19:45:00 Alpha in sample price insight for xlf       

### 5. Seach Alpha Streams by Author Details
Knowing who created the Alpha can be an important search tool to improve your filtering. Participating Author information can be searched in a similar manner to the Alpha Search. Each supplied criteria is a logical AND on the query.


In [11]:
languages = ["Py"]

authors = client.SearchAuthors( languages = languages, location = "New York" )

print("Located {} authors matching your criteria.".format(len(authors)))

for a in authors[:5]:
    print("Alpha Stream Author {:>5} is from {}. Signed up {} and codes in {}".format( a.Id[:5], a.Location, a.SignUpTime, a.Language) )

Located 5 authors matching your criteria.
Alpha Stream Author 1675c is from New York, New York, US. Signed up 2018-01-30 23:50:50 and codes in Py
Alpha Stream Author 3c516 is from New York, New York, US. Signed up 2018-04-02 03:28:59 and codes in Py
Alpha Stream Author 708e6 is from Brooklyn, New York, US. Signed up 2017-04-13 02:22:31 and codes in Py
Alpha Stream Author 73cb0 is from Hicksville, New York, US. Signed up 2018-04-09 22:29:27 and codes in Py
Alpha Stream Author 7ac7a is from Brooklyn, New York, US. Signed up 2017-04-25 13:17:20 and codes in Py


In [12]:
import matplotlib.pyplot as plt
sizes = [len(client.SearchAuthors( languages = "Py")), len(client.SearchAuthors( languages = "C#"))]
plt.figure(figsize=(10,5))
plt.pie(sizes, explode=(0.1, 0), labels= ['Python', 'C#'], colors=['lightgrey', 'orange'], autopct='%1.1f%%', startangle=140)
plt.axis('equal')
plt.show()

<Figure size 1000x500 with 1 Axes>

### 6. Fetch Detailed Information on a Specific Author
The API allows you to fetch detailed information on a specific author by the hash-id.

In [13]:
# Author to fetch profile information about:
authorId = "1f48359f6c6cbad65b091232eaae73ce"

# Submit request for more information:
author = client.GetAuthorByIdRequest(authorId)

# Print out the Alpha data:
print( "Author Matched Id: \n\t id: {} \n\t Biography: {}... \n\t Projects: {} \n\t Language: {} \n\t Signed Up: {} \n\t Location: {} ".
      format(author.Id[:5], author.Biography[:50], author.Projects, author.Language, author.SignUpTime, author.Location ) )

Author Matched Id: 
	 id: 1f483 
	 Biography: I'm part of the QuantConnect Engineering team. I h... 
	 Projects: 537 
	 Language: C# 
	 Signed Up: 2015-03-28 20:10:29 
	 Location: New Hope, Pennsylvania, US 


### 7. Subscribing and Unsubscribing from An Alpha
Once you've located an Alpha you need to subscribe to it to recieve its streaming insights. Alphas have a monthly licensing fee which is engaged once you request its signals. There is a minimum fee equivalent to 7 days of licensing from an Alpha. Sending a subscription request starts delivery of the signals to the fund-exchange.

In [18]:
# Alpha to subscribe to:
alphaId = "392a40ccab3740287a1c30bc6"

unsubscribeResult = client.Unsubscribe( alphaId )
print("Stopped Subscription to {}: {}".format(alphaId, unsubscribeResult))

Stopped Subscription to 392a40ccab3740287a1c30bc6: True


In [17]:
subscribeResult = client.Subscribe( alphaId )
print("Started Subscription to {}: {}".format(alphaId, subscribeResult))

Started Subscription to 392a40ccab3740287a1c30bc6: True


### 8. Stream Insights from An Alpha
Finally once analysed and subscribed to an Alpha - you can pull down the individual insights with the streaming socket connection. This is to use the insights in realtime in your fund management.

In [19]:
# Import the streaming client helper library
from AlphaInsightsStreamClient import AlphaInsightsStreamClient

# Configure authentication details
ipaddress = "35.231.13.1"
user = "demo-api"
password = "demo"
virtualhost = "demo-client"
exchange = "QCAlphaExchange_Demo-Client"
queueName = alphaId
routingKey = alphaId

# Create the streaming client tool:
streamClient = AlphaInsightsStreamClient(user, password, ipaddress, virtualhost, exchange)

print("Streaming insights for 5 seconds... ")

# StreamSynchronously pulls down Insights for a minimum timeout in a blocking manner.
for i in streamClient.StreamSynchronously( alphaId, timeout=10 ):
    print( "{} Alpha {} {} insight for {:<10} going {} by {:.2%} over the next {}s".format(i.Created, i.Source, i.Type, i.Ticker, i.Direction, i.Magnitude, i.Period) )

print("Streaming completed.")
    

Streaming insights for 5 seconds... 
Streaming completed.


In [20]:
alphaId = "392a40ccab3740287a1c30bc6"

prices = client.GetAlphaPrices(alphaId, start = 0)

# Take Top 5:
for i in prices[:10]:
    print( "{} price from {}, shared ${:,.2f} and the exclusive ${:,.2f}".format(i.PriceType.title(), i.Time, i.SharedPrice, i.ExclusivePrice) )

Ask price from 2018-07-23 16:53:04, shared $104,000.00 and the exclusive $104,000.00


In [21]:
alphaId = "392a40ccab3740287a1c30bc6"

errors = client.GetAlphaErrors(alphaId, start = 0)

# Take Top 5:
for i in errors[:10]:
    print( "Error at {}, error message: {}, stacktrace: {}".format(i.Time, i.Error[:10], i.Stacktrace[:10]) ) 

Error at 2018-07-30 21:30:58, error message: Test Error, stacktrace: Test stack
Error at 2018-07-30 21:35:48, error message: Test Error, stacktrace: Test stack
Error at 2018-07-30 21:35:49, error message: Test Error, stacktrace: Test stack
